In [1]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [8]:
!streamlit hello

"streamlit" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
